In [2]:
import cv2
import numpy as np
from tkinter import Tk, Button, filedialog, Canvas
from PIL import Image, ImageTk

import warnings

warnings.filterwarnings("ignore")


# Function to handle button click and image stitching
def stitch_images():
    try:
        # Open file dialog to select the first image
        image1_path = filedialog.askopenfilename(title="Select the first image")

        # Open file dialog to select the second image
        image2_path = filedialog.askopenfilename(title="Select the second image")

        # Read input images
        image1 = cv2.imread(image1_path)
        image2 = cv2.imread(image2_path)

        # Create a stitcher object
        stitcher = cv2.Stitcher_create()

        # Stitch the images
        status, stitched_image = stitcher.stitch([image1, image2])

        if status == cv2.Stitcher_OK:
            # Convert the BGR image to RGB for displaying in tkinter
            stitched_image_rgb = cv2.cvtColor(stitched_image, cv2.COLOR_BGR2RGB)
            # Convert the RGB image to PIL format
            pil_image = Image.fromarray(stitched_image_rgb)
            # Resize the image to fit the canvas
            pil_image = pil_image.resize((canvas_width, canvas_height), Image.ANTIALIAS)
            # Create a PhotoImage object from the PIL image
            photo_image = ImageTk.PhotoImage(pil_image)

            # Clear existing image on the canvas
            canvas.delete("image")
            # Create a Canvas image item and display the stitched image
            canvas.create_image(0, 0, anchor="nw", image=photo_image, tags="image")
            # Keep a reference to the photo image to prevent it from being garbage collected
            canvas.image = photo_image

        else:
            print("Image stitching failed.")

    except Exception as e:
        print("An error occurred:", str(e))


# Create a simple GUI using tkinter
root = Tk()

# Rename the Tkinter window
root.title("Image Stitching")

# Set the window size and position it in the center of the screen
window_width = 800
window_height = 500
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
window_x = (screen_width - window_width) // 2
window_y = (screen_height - window_height) // 2
root.geometry(f"{window_width}x{window_height}+{window_x}+{window_y}")

# Configure the file dialog appearance
root.option_add("*Dialog.msg.width", 20)  # Set width
root.option_add("*Dialog.msg.wrapLength", window_width // 2)  # Set wrap length
root.option_add("*Dialog.msg.font", "Helvetica 12")  # Set font

# Customize the file dialog window
root.tk_setPalette(background="white")

# Calculate the canvas size
canvas_width = window_width
canvas_height = window_height - 50

# Create a button to choose images
button = Button(root, text="Choose Images", command=stitch_images)
button.grid(row=0, column=0, padx=10, pady=10)

# Create a Canvas widget
canvas = Canvas(root, width=canvas_width, height=canvas_height)
canvas.grid(row=1, column=0, padx=10, pady=10)


# Function to handle window close event
def on_close():
    root.quit()  # Stop the main loop
    root.destroy()  # Destroy the root window


root.protocol("WM_DELETE_WINDOW", on_close)  # Call on_close when the window is closed
root.mainloop()